# Capítulo V: Agentes

In [1]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
#Se instancia la herramienta de búsqueda web
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
]

In [4]:
llm=OpenAI(temperature=0)

In [5]:
from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain import hub

In [ ]:
#PromptTemplate: hwchase17/react-chat
"""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

In [7]:
prompt = hub.pull("hwchase17/react-chat")

In [15]:
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [17]:
llm_with_stop = llm.bind(stop=["\nObservation"])

In [18]:
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_stop | ReActSingleInputOutputParser()

In [19]:
from langchain.agents import AgentExecutor

In [20]:
memory = ConversationBufferMemory(memory_key="chat_history")
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [21]:
agent_executor.invoke({"input": "Hola! Mi nombre es Joaquin"})['output']



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
Final Answer: Hola Joaquin, ¡encantado de conocerte! ¿En qué puedo ayudarte?

> Finished chain.


'Hola Joaquin, ¡encantado de conocerte! ¿En qué puedo ayudarte?'

In [22]:
agent_executor.invoke({"input": "Sabes cómo me llamo?"})['output']



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
Final Answer: Sí, ya te dije que mi nombre es Joaquin. ¿En qué más puedo ayudarte?

> Finished chain.


'Sí, ya te dije que mi nombre es Joaquin. ¿En qué más puedo ayudarte?'

In [23]:
agent_executor.invoke({"input": "¿Cual fue el marcador del último partido de Colo Colo como local?"})['output']



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: último partido de Colo Colo como local{'title': 'Colo-Colo', 'rankings': '3rd in Chilean Primera División', 'thumbnail': 'https://serpapi.com/searches/651337133c3fb215e7da4da5/images/75b9dfc42bf701752d1de1fab635cd51550418d8a2dffbc1cc9b779ad9aa157f.png', 'game_spotlight': {'league': 'Chilean Primera División', 'stadium': 'Estadio Monumental David Arellano', 'date': 'Sat, Sep 23', 'stage': 'Full-time', 'teams': [{'name': 'Colo-Colo', 'thumbnail': 'https://serpapi.com/searches/651337133c3fb215e7da4da5/images/75b9dfc42bf701758ee74cd813b7ccf3cc8fd073f896289e93323494b218db922fb2af06b2f9a6268a42598f64143caf.png', 'score': '6'}, {'name': 'Cobresal', 'thumbnail': 'https://serpapi.com/searches/651337133c3fb215e7da4da5/images/75b9dfc42bf701758ee74cd813b7ccf3cc8fd073f896289e93323494b218db9289a5a6a16df35e6c012b8cacab33b782.png', 'score': '0'}]}} Do I need to use a tool? No
Final Answe

'El último partido de Colo Colo como local fue contra Cobresal, el marcador fue 6-0 a favor de Colo Colo.'